In [ ]:
from algorithms import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from aux_functions import *
import statistics
from sklearn.metrics import mean_absolute_error

In [ ]:
house = pd.read_csv("C:/Users/FEEL/Documents/Datasets/REFIT/Processed_Data_CSV/House_1.csv")
house.index = pd.to_datetime(house["Time"])
house.drop("Time", axis = 1, inplace = True)
house = house.resample("1min").mean()
house.dropna(inplace = True)
house = house.loc['2014-06-20':'2015-06-20']

xtrain = house.iloc[:int(0.66*house.shape[0])]["Aggregate"] 
xtest = house.iloc[int(0.66*house.shape[0]):]["Aggregate"]
ytrain = house.iloc[:int(0.66*house.shape[0])]["Appliance5"]
ytest = house.iloc[int(0.66*house.shape[0]):]["Appliance5"]

In [ ]:
#WGRU
network = WindowGRU(xtrain, ytrain, window_size=10, model_name="exp1_wgru", 
                       model_dir = "", n_epochs=5, stop_patience=2, 
                       dropout_rate=0.2, batch_size=128, 
                       use_callbacks=True)

In [ ]:
#AttentionRNN
network = AttentionRNN(xtrain, ytrain, window_size=10, model_name="exp1_att", 
                       model_dir = "", n_epochs=5, stop_patience=2, 
                       dropout_rate=0.2, batch_size=128, 
                       use_callbacks=True)

In [ ]:
network.fit()

In [ ]:
ws, inic, fim = 10, '23:00', '00:00'
xtest1 = xtest.between_time(inic, fim)
ytest1 = ytest.between_time(inic, fim)

In [ ]:
#AttentionRNN
results = network.predict(xtest1)
results = results.flatten()
ytest_star = ytest1[ws:]
mean_absolute_error(ytest_star, results)

In [ ]:
#WGRU
results = network.predict(xtest1)
results_star =  [results[i][0][0] for i in range(0,results.shape[0])] 
ytest_star = ytest1[ws:]
mean_absolute_error(ytest_star, results_star)